In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [4]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

USER_AGENT environment variable not set, consider setting it to identify your requests.


43131

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

66

In [8]:
all_splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 7056}

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings

# Initialize embedding with HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

e:\D Drive\Minor Project\MemoPin\ai-backend\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [11]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(documents=all_splits, embedding=embeddings)


In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})



6

In [14]:
### Lets Put together the whole pipeline

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")


Failed to get info from https://api.smith.langchain.com: LangSmithConnectionError('Connection error caused failure to GET /info in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError(\'HTTPSConnectionPool(host=\\\'api.smith.langchain.com\\\', port=443): Max retries exceeded with url: /info (Caused by ReadTimeoutError("HTTPSConnectionPool(host=\\\'api.smith.langchain.com\\\', port=443): Read timed out. (read timeout=10.0)"))\'))\nContent-Length: None\nAPI Key: lsv2_********************************************f6')
Failed to get info from https://api.smith.langchain.com: LangSmithConnectionError('Connection error caused failure to GET /info in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError(\'HTTPSConnectionPool(host=\\\'api.smith.langchain.com\\\', port=443): Max retries exceeded with url: /info (Caused by ReadTimeoutError("HTTPSConnectionPool(host=\\\'api.smith.langchain.com\\\', port=443): Read timed out. (read tim

Failed to get info from https://api.smith.langchain.com: LangSmithConnectionError('Connection error caused failure to GET /info in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError(\'HTTPSConnectionPool(host=\\\'api.smith.langchain.com\\\', port=443): Max retries exceeded with url: /info (Caused by ReadTimeoutError("HTTPSConnectionPool(host=\\\'api.smith.langchain.com\\\', port=443): Read timed out. (read timeout=10.0)"))\'))\nContent-Length: None\nAPI Key: lsv2_********************************************f6')


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]

In [20]:
import time
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

def fade_in_text(text, delay=0.05):
    for char in text:
        print(char, end="", flush=True)
        time.sleep(delay)  # Controls the speed of the fading effect

# Stream output with a fading effect
for chunk in rag_chain.stream("What is Task Decomposition?"):
    fade_in_text(chunk)
    time.sleep(0.1)  # Delay between chunks


Task Decomposition refers to the process of breaking down a complicated task into smaller and simpler steps, allowing for more manageable and interpretable planning. This can be achieved through techniques like Chain of Thought (CoT) and Tree of Thoughts, which involve instructing the model to think step-by-step or exploring multiple reasoning possibilities at each step.